In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle

import util

dir_source_data = "data/source_data_files"


In [2]:
"""
Fig. 2 (aggregate measures of task performance and human-model similarity)
"""
# These files contain results from all model experiments
fn_data_localization = "data/model/sound_localization.pkl"
fn_data_spkr_word = "data/model/spkr_word_recognition.pkl"
# These files contain human-model comparison metrics for all experiments
fn_comparison_localization = "data/model/sound_localization_human_model_comparison_metrics.pkl"
fn_comparison_spkr_word = "data/model/spkr_word_recognition_human_model_comparison_metrics.pkl"

dict_tag_model = {
    "localization": {
        "3000_delayed": "models/sound_localization/IHC3000_delayed_integration/arch??",
        "3000": "models/sound_localization/IHC3000/arch??",
        "1000": "models/sound_localization/IHC1000/arch??",
        "320": "models/sound_localization/IHC0320/arch??",
        "50": "models/sound_localization/IHC0050/arch??",
        "3000_delayed_simple": "models/sound_localization/simplified_IHC3000_delayed_integration/arch??",
        "3000_simple": "models/sound_localization/simplified_IHC3000/arch??",
        "1000_simple": "models/sound_localization/simplified_IHC1000/arch??",
        "320_simple": "models/sound_localization/simplified_IHC0320/arch??",
        "50_simple": "models/sound_localization/simplified_IHC0050/arch??",
    },
    "spkr_word": {
        "3000": "models/spkr_word_recognition/IHC3000/arch0_00??",
        "1000": "models/spkr_word_recognition/IHC1000/arch0_00??",
        "320": "models/spkr_word_recognition/IHC0320/arch0_00??",
        "50": "models/spkr_word_recognition/IHC0050/arch0_00??",
        "3000_simple": "models/spkr_word_recognition/simplified_IHC3000/arch0_00??",
        "1000_simple": "models/spkr_word_recognition/simplified_IHC1000/arch0_00??",
        "320_simple": "models/spkr_word_recognition/simplified_IHC0320/arch0_00??",
        "50_simple": "models/spkr_word_recognition/simplified_IHC0050/arch0_00??",
    },
    "spkr": {
        "3000": "models/spkr_recognition/IHC3000/arch0_00??",
        "1000": "models/spkr_recognition/IHC1000/arch0_00??",
        "320": "models/spkr_recognition/IHC0320/arch0_00??",
        "50": "models/spkr_recognition/IHC0050/arch0_00??",
    },
    "word": {
        "3000": "models/word_recognition/IHC3000/arch0_00??",
        "1000": "models/word_recognition/IHC1000/arch0_00??",
        "320": "models/word_recognition/IHC0320/arch0_00??",
        "50": "models/word_recognition/IHC0050/arch0_00??",
    }
}

list_key_metric = ["performance", "pearsonr", "rmse"]
list_key_task = [
    "localization",
    "spkr",
    "word",
]
list_key_model = [
    "50",
    "320",
    "1000",
    "3000",
    "3000_delayed",
]
df = {
    "metric": [],
    "task": [],
    "model": [],
    "y_mean": [],
    "y_bootstrap": [],
}
for key_metric in list_key_metric:
        for key_task in list_key_task:
            for key_model in list_key_model:
                if key_task in ["spkr", "word"]:
                    tag_model = dict_tag_model["spkr_word"].get(key_model, None)
                else:
                    tag_model = dict_tag_model["localization"].get(key_model, None)
                if tag_model is not None:
                    y, list_y, bootstrap_list_y = util.get_aggregate_measure(
                        fn_data_localization=fn_data_localization,
                        fn_data_spkr_word=fn_data_spkr_word,
                        fn_comparison_localization=fn_comparison_localization,
                        fn_comparison_spkr_word=fn_comparison_spkr_word,
                        list_tag_model=[tag_model],
                        key_task=key_task,
                        key_metric=key_metric,
                    )
                    assert bootstrap_list_y.shape[0] == 1
                    bootstrap_list_y = list(bootstrap_list_y.reshape([-1]))
                    assert y.shape == (1,)
                    y = y[0]
                    n = len(bootstrap_list_y)
                    df["metric"].extend([key_metric] * n)
                    df["task"].extend([key_task] * n)
                    df["model"].extend([key_model] * n)
                    df["y_mean"].extend([y] * n)
                    df["y_bootstrap"].extend(bootstrap_list_y)
                    print(key_metric, key_task, key_model, len(df["y_bootstrap"]))
df = pd.DataFrame(df)
df.to_csv(os.path.join(dir_source_data, "fig2_fig4e.csv"), index=False)


performance localization 50 1000
performance localization 320 2000
performance localization 1000 3000
performance localization 3000 4000
performance localization 3000_delayed 5000
performance spkr 50 6000
performance spkr 320 7000
performance spkr 1000 8000
performance spkr 3000 9000
performance word 50 10000
performance word 320 11000
performance word 1000 12000
performance word 3000 13000
pearsonr localization 50 14000
pearsonr localization 320 15000
pearsonr localization 1000 16000
pearsonr localization 3000 17000
pearsonr localization 3000_delayed 18000
pearsonr spkr 50 19000
pearsonr spkr 320 20000
pearsonr spkr 1000 21000
pearsonr spkr 3000 22000
pearsonr word 50 23000
pearsonr word 320 24000
pearsonr word 1000 25000
pearsonr word 3000 26000
rmse localization 50 27000
rmse localization 320 28000
rmse localization 1000 29000
rmse localization 3000 30000
rmse localization 3000_delayed 31000
rmse spkr 50 32000
rmse spkr 320 33000
rmse spkr 1000 34000
rmse spkr 3000 35000
rmse word 5

In [3]:
"""
Load sound localization data
"""
fn_data = "data/model/sound_localization.pkl"
with open(fn_data, "rb") as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
map_k_to_tag_model = {
    "human": "human",
    "3000_delayed": "models/sound_localization/IHC3000_delayed_integration/arch??",
    "3000": "models/sound_localization/IHC3000/arch??",
    "1000": "models/sound_localization/IHC1000/arch??",
    "320": "models/sound_localization/IHC0320/arch??",
    "50": "models/sound_localization/IHC0050/arch??",
    "3000_delayed_simple": "models/sound_localization/simplified_IHC3000_delayed_integration/arch??",
    "3000_simple": "models/sound_localization/simplified_IHC3000/arch??",
    "1000_simple": "models/sound_localization/simplified_IHC1000/arch??",
    "320_simple": "models/sound_localization/simplified_IHC0320/arch??",
    "50_simple": "models/sound_localization/simplified_IHC0050/arch??",
}
map_tag_model_to_k = {v: k for k, v in map_k_to_tag_model.items()}


In [4]:
"""
Fig. 3d (Sound localization performance as a function of SNR)
"""
df = EXPERIMENT_DATAFRAMES["snr_dependency"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"})

def add_subject_list(dfi):
    key_metric_list = "deg_err_list"
    if dfi.model == "human":
        dfi.subject_list = ["S{}".format(_) for _ in range(1, len(dfi[key_metric_list]) + 1)]
    else:
        dfi.subject_list = ["arch{:02.0f}".format(_) for _ in range(1, len(dfi[key_metric_list]) + 1)]
    return dfi

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df.to_csv(os.path.join(dir_source_data, "fig3d.csv"), index=False)
df


,deg_azim_err,deg_azim_err_mean,deg_azim_err_sem,deg_elev_err,deg_elev_err_mean,deg_elev_err_sem,deg_err,deg_err_mean,deg_err_sem,snr,model,subject
0,37.662338,38.457715,1.181336,15.324675,15.069906,0.445382,40.270846,41.589671,0.849888,-13.6,human,S1
1,47.236842,38.457715,1.181336,14.473684,15.069906,0.445382,45.982487,41.589671,0.849888,-13.6,human,S2
2,39.220779,38.457715,1.181336,16.883117,15.069906,0.445382,43.847728,41.589671,0.849888,-13.6,human,S3
3,36.103896,38.457715,1.181336,15.324675,15.069906,0.445382,39.124571,41.589671,0.849888,-13.6,human,S4
4,42.207792,38.457715,1.181336,12.857143,15.069906,0.445382,42.976065,41.589671,0.849888,-13.6,human,S5
...,...,...,...,...,...,...,...,...,...,...,...,...
301,2.25492,1.789039,0.152206,5.117162,4.097437,0.217833,6.497908,5.178754,0.301360,inf,3000,arch06
302,1.87254,1.789039,0.152206,4.395881,4.097437,0.217833,5.531723,5.178754,0.301360,inf,3000,arch07
303,2.407094,1.789039,0.152206,4.6,4.097437,0.217833,6.069171,5.178754,0.301360,inf,3000,arch08
304,1.275744,1.789039,0.152206,3.021739,4.097437,0.217833,3.828746,5.178754,0.301360,inf,3000,arch09


In [5]:
"""
Fig. 3f (ITD and ILD perceptual cue weighting)
"""
df = EXPERIMENT_DATAFRAMES["itd_ild_weighting"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"})

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})

def fn_to_subject(fn):
    if "/" in fn:
        subject = [_ for _ in fn.split("/") if "arch" in _][0]
    else:
        subject = fn
    return subject

df["subject"] = df["fn_eval"].map(fn_to_subject)
df["band"] = df["band"].map({
    "(4000, 16000)": "highpass",
    "(500, 2000)": "lowpass",
})
df = df[~df["band"].isna()].rename(columns={"band": "condition"})
df = df.drop(columns=[
    "ild_bias_imposed",
    "ild_bias_response",
    "itd_bias_imposed",
    "itd_bias_response",
    "fn_eval",
])
df.to_csv(os.path.join(dir_source_data, "fig3f.csv"), index=False)
df


,condition,ild_bias_weight,ild_bias_weight_mean,ild_bias_weight_sem,itd_bias_weight,itd_bias_weight_mean,itd_bias_weight_sem,model,subject
0,highpass,0.773329,0.818837,0.042054,0.297845,0.237883,0.049826,human,S66
1,highpass,0.930587,0.818837,0.042054,0.243525,0.237883,0.049826,human,S67
2,highpass,0.792194,0.818837,0.042054,0.128392,0.237883,0.049826,human,S68
3,highpass,0.970695,0.818837,0.042054,0.192826,0.237883,0.049826,human,S69
4,highpass,0.95758,0.818837,0.042054,0.19284,0.237883,0.049826,human,S74
...,...,...,...,...,...,...,...,...,...
145,lowpass,0.260585,0.264845,0.014405,1.00327,0.985585,0.006180,3000,arch06
146,lowpass,0.2287,0.264845,0.014405,0.964051,0.985585,0.006180,3000,arch07
147,lowpass,0.259876,0.264845,0.014405,1.006623,0.985585,0.006180,3000,arch08
148,lowpass,0.284708,0.264845,0.014405,1.008132,0.985585,0.006180,3000,arch09


In [6]:
"""
Fig. 3h (Minimum audible angle thresholds)
"""
df = EXPERIMENT_DATAFRAMES["maa_azimuth"]
list_k_model = [
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"})

def add_subject_list(dfi):
    if dfi.model == "human":
        dfi.subject_list = ["S1_250Hz", "S1_500Hz", "S1_750Hz", "S1_1000Hz"]
    else:
        dfi.subject_list = ["arch{:02.0f}".format(_) for _ in range(1, len(dfi["maa_list"]) + 1)]
    return dfi

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)
keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df = df.drop(columns=["maa_mean", "maa_sem"])

df_ref = pd.read_csv("data/human/sound_localization/mills_1958_minimum_audible_angle.csv").assign(model="human")
df_ref = df_ref[df_ref.f <= 1e3]
df_ref = df_ref.rename(columns={"azim": "azim_ref", "f": "subject"})
df_ref["subject"] = df_ref["subject"].map(lambda _: "S1_{:04d}Hz".format(_))
df_ref["log_maa"] = np.log(df_ref["maa"])
df_ref = util.flatten_columns(df_ref.groupby(["azim_ref", "model"]).agg({
    "maa": list,
    "log_maa": [list, "mean", "sem"],
    "subject": list,
}).reset_index(), sep="_")
keys_to_explode = [c for c in df_ref.columns if c.endswith("_list")]
df_ref = df_ref.explode(keys_to_explode).reset_index(drop=True)
df_ref = df_ref.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})

df = pd.concat([df_ref, df])
df.to_csv(os.path.join(dir_source_data, "fig3h.csv"), index=False)
df


,azim_ref,model,maa,log_maa,log_maa_mean,log_maa_sem,subject
0,0,human,1.20616,0.187441,0.075947,0.070967,S1_0250Hz
1,0,human,1.010851,0.010792,0.075947,0.070967,S1_0500Hz
2,0,human,0.910907,-0.093315,0.075947,0.070967,S1_0750Hz
3,0,human,1.220021,0.198868,0.075947,0.070967,S1_1000Hz
4,30,human,1.698859,0.529957,0.491413,0.028157,S1_0250Hz
...,...,...,...,...,...,...,...
635,75,50,90.0,4.49981,4.220840,0.065572,arch06
636,75,50,73.623919,4.235822,4.220840,0.065572,arch07
637,75,50,90.0,4.49981,4.220840,0.065572,arch08
638,75,50,64.106675,4.071465,4.220840,0.065572,arch09


In [7]:
"""
Fig. 4b and 4d (ITD lateralization thresholds)
"""
df = EXPERIMENT_DATAFRAMES["itd_threshold"]
list_k_model = [
    "human",
    "50",
    "320",
    "1000",
    "3000",
    "3000_delayed",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"})

def add_subject_list(dfi):
    key_metric_list = "log_itd_threshold_list"
    if dfi.model == "human":
        dfi.subject_list = ["S{}".format(_) for _ in range(1, len(dfi[key_metric_list]) + 1)]
    else:
        dfi.subject_list = ["arch{:02.0f}".format(_) for _ in range(1, len(dfi[key_metric_list]) + 1)]
    return dfi

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df.to_csv(os.path.join(dir_source_data, "fig4b_fig4d.csv"), index=False)
df


,f,itd_threshold,itd_threshold_mean,itd_threshold_sem,log_itd_threshold,log_itd_threshold_mean,log_itd_threshold_sem,model,subject
0,250.0,61.0,64.0,12.369317,4.110874,4.101844,0.196802,human,S1
1,250.0,61.0,64.0,12.369317,4.110874,4.101844,0.196802,human,S2
2,250.0,37.0,64.0,12.369317,3.610918,4.101844,0.196802,human,S3
3,250.0,97.0,64.0,12.369317,4.574711,4.101844,0.196802,human,S4
4,500.0,31.0,34.5,6.291529,3.433987,3.497613,0.163847,human,S1
...,...,...,...,...,...,...,...,...,...
3035,3200.0,320.0,320.0,0.000000,5.768321,5.768321,0.000000,50,arch06
3036,3200.0,320.0,320.0,0.000000,5.768321,5.768321,0.000000,50,arch07
3037,3200.0,320.0,320.0,0.000000,5.768321,5.768321,0.000000,50,arch08
3038,3200.0,320.0,320.0,0.000000,5.768321,5.768321,0.000000,50,arch09


In [8]:
"""
Fig. 7e (speech localization in anechoic / reverberant conditions)
"""
df = EXPERIMENT_DATAFRAMES["speech_in_noise_in_reverb"]
list_k_model = [
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"})
df = df[df["index_room"].isin([1, 0]) & np.isfinite(df["snr"])]

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df["subject"] = df["fn_eval"].map(lambda _: [x for x in _.split("/") if "arch" in x][0])
df["condition"] = df["index_room"].map({0: "reverberant", 1: "anechoic"})
df = df[[
    "condition",
    "snr",
    "correct",
    "correct_mean",
    "correct_sem",
    "model",
    "subject",
]]

df.to_csv(os.path.join(dir_source_data, "fig7e.csv"), index=False)
df


,condition,snr,correct,correct_mean,correct_sem,model,subject
0,reverberant,-24.0,0.087766,0.082181,0.002583,50,arch01
1,reverberant,-24.0,0.085106,0.082181,0.002583,50,arch02
2,reverberant,-24.0,0.077128,0.082181,0.002583,50,arch03
3,reverberant,-24.0,0.06383,0.082181,0.002583,50,arch04
4,reverberant,-24.0,0.082447,0.082181,0.002583,50,arch05
...,...,...,...,...,...,...,...
715,anechoic,24.0,1.0,1.000000,0.000000,3000,arch06
716,anechoic,24.0,1.0,1.000000,0.000000,3000,arch07
717,anechoic,24.0,1.0,1.000000,0.000000,3000,arch08
718,anechoic,24.0,1.0,1.000000,0.000000,3000,arch09


In [9]:
"""
Fig. 7f (speech localization thresholds in anechoic / reverberant conditions)
"""
df = pd.read_csv(os.path.join(dir_source_data, "fig7e.csv"))
df = df.groupby(["condition", "model", "subject"]).agg({
    "snr": list,
    "correct": list,
}).reset_index()

def get_threshold_snr(dfi):
    x = np.array(dfi.snr)
    y = np.array(dfi.correct)
    y = y[~np.isinf(x)]
    x = x[~np.isinf(x)]
    return np.interp(0.707, y, x)

df["threshold"] = df.apply(get_threshold_snr, axis=1)
df = df[[
    "condition",
    "threshold",
    "model",
    "subject",
]]

df.to_csv(os.path.join(dir_source_data, "fig7f.csv"), index=False)
df


,condition,threshold,model,subject
0,anechoic,-4.758330,50,arch01
1,anechoic,-5.094212,50,arch02
2,anechoic,-4.546605,50,arch03
3,anechoic,-0.858162,50,arch04
4,anechoic,-4.494821,50,arch05
...,...,...,...,...
75,reverberant,-6.521436,3000,arch06
76,reverberant,-1.816235,3000,arch07
77,reverberant,-5.646277,3000,arch08
78,reverberant,-7.736064,3000,arch09


In [10]:
"""
Load word and voice recognition data
"""
fn_data = "/om2/user/msaddler/phaselocknet/data/model/spkr_word_recognition.pkl"
with open(fn_data, "rb") as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
map_k_to_tag_model = {
    "human": "human",
    "3000": "models/spkr_word_recognition/IHC3000/arch0_00??",
    "1000": "models/spkr_word_recognition/IHC1000/arch0_00??",
    "320": "models/spkr_word_recognition/IHC0320/arch0_00??",
    "50": "models/spkr_word_recognition/IHC0050/arch0_00??",
    "3000_simple": "models/spkr_word_recognition/simplified_IHC3000/arch0_00??",
    "1000_simple": "models/spkr_word_recognition/simplified_IHC1000/arch0_00??",
    "320_simple": "models/spkr_word_recognition/simplified_IHC0320/arch0_00??",
    "50_simple": "models/spkr_word_recognition/simplified_IHC0050/arch0_00??",
    "3000_spkr": "models/spkr_recognition/IHC3000/arch0_00??",
    "1000_spkr": "models/spkr_recognition/IHC1000/arch0_00??",
    "320_spkr": "models/spkr_recognition/IHC0320/arch0_00??",
    "50_spkr": "models/spkr_recognition/IHC0050/arch0_00??",
    "3000_word": "models/word_recognition/IHC3000/arch0_00??",
    "1000_word": "models/word_recognition/IHC1000/arch0_00??",
    "320_word": "models/word_recognition/IHC0320/arch0_00??",
    "50_word": "models/word_recognition/IHC0050/arch0_00??",
}
map_tag_model_to_k = {v: k for k, v in map_k_to_tag_model.items()}

def add_subject_list(dfi, key_metric_list="correct_word_list"):
    if dfi.model == "human":
        if isinstance(dfi[key_metric_list], list):
            dfi.subject_list = list(range(len(dfi[key_metric_list])))
        else:
            dfi[key_metric_list] = [dfi[key_metric_list]]
            dfi.subject_list = [None]
    else:
        subject_list = [
            "arch0_0000",
            "arch0_0001",
            "arch0_0002",
            "arch0_0004",
            "arch0_0006",
            "arch0_0007",
            "arch0_0008",
            "arch0_0009",
            "arch0_0016",
            "arch0_0017",
        ]
        dfi.subject_list = [subject_list[_] for _ in range(len(dfi[key_metric_list]))]
    return dfi


In [11]:
"""
Fig. 5b and 5c (word and voice recognition as a function of SNR and noise condition)
"""
df = EXPERIMENT_DATAFRAMES["kell_like"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model", "background_condition": "condition"}).drop(columns=["tag_expt"])
df["condition"] = df["condition"].map({
    0.0: "auditory_scenes",
    1.0: "8_talker_babble",
    2.0: "instrumental_music",
    3.0: "stationary_noise",
})
df = df[~df.condition.isna()]
df = df[df.snr.isin([-9, -6, -3, 0, 3, np.inf])]

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)
df.loc[df["model"] == "human", "correct_spkr_list"] = df.loc[df["model"] == "human", "correct_word_list"]

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df.loc[df["model"] == "human", "correct_spkr"] = df.loc[df["model"] == "human", "correct_spkr_mean"]
df = df[[
    "condition",
    "snr",
    "correct_word",
    "correct_word_mean",
    "correct_word_sem",
    "correct_spkr",
    "correct_spkr_mean",
    "correct_spkr_sem",
    "model",
    "subject",
]]
df.to_csv(os.path.join(dir_source_data, "fig5b_fig5c.csv"), index=False)
df


,condition,snr,correct_word,correct_word_mean,correct_word_sem,correct_spkr,correct_spkr_mean,correct_spkr_sem,model,subject
0,auditory_scenes,-9.0,0.066667,0.171557,0.015466,NaN,NaN,NaN,human,0
1,auditory_scenes,-9.0,0.071429,0.171557,0.015466,NaN,NaN,NaN,human,1
2,auditory_scenes,-9.0,0.1,0.171557,0.015466,NaN,NaN,NaN,human,2
3,auditory_scenes,-9.0,0.285714,0.171557,0.015466,NaN,NaN,NaN,human,3
4,auditory_scenes,-9.0,0.071429,0.171557,0.015466,NaN,NaN,NaN,human,4
...,...,...,...,...,...,...,...,...,...,...
2011,stationary_noise,inf,0.875,0.866223,0.003865,0.978723,0.979521,0.001819,3000,arch0_0007
2012,stationary_noise,inf,0.869681,0.866223,0.003865,0.992021,0.979521,0.001819,3000,arch0_0008
2013,stationary_noise,inf,0.851064,0.866223,0.003865,0.978723,0.979521,0.001819,3000,arch0_0009
2014,stationary_noise,inf,0.853723,0.866223,0.003865,0.976064,0.979521,0.001819,3000,arch0_0016


In [12]:
"""
Fig. 5e and 5f (word and voice recognition in 43 different auditory textures)
"""
df = EXPERIMENT_DATAFRAMES["speech_in_synthetic_textures"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model", "index_texture": "condition"}).drop(columns=["tag_expt"])
map_condition = {
    0: "16mm film projector",
    1: "Airplane engine",
    2: "Airplane cabin",
    3: "Alarm bell",
    4: "Applause (large crowd)",
    5: "Applause (small crowd)",
    6: "Bamboo water fountain",
    7: "Bicycle",
    8: "Birdsong",
    9: "Disc grinder",
    10: "Dolphins clicking",
    11: "Fire (small, indoor)",
    12: "Fire (large, outdoor)",
    13: "Fire (dry grass, near)",
    14: "Fire (windy, distant)",
    15: "Frying bacon",
    16: "Hand drill",
    17: "Heavy traffic (wet road)",
    18: "Hissing air (small)",
    19: "Hissing air (large)",
    20: "Jackhammer",
    21: "Jungle environment",
    22: "Manual hand mixer",
    23: "Mechanical - castanets",
    24: "Mechanical - frying bacon",
    25: "Mechanical - horse trotting",
    26: "Mechanical - typewriter",
    27: "Mechanical - linotypes",
    28: "Mechanical - lathe",
    29: "Mechanical - shaking coins",
    30: "Military jet engine",
    31: "People clapping (theater)",
    32: "People clapping (crowd)",
    33: "Rain sounds in a car",
    34: "Slot machine",
    35: "Summer night insects",
    36: "Typing",
    37: "Thunder and rain",
    38: "Water (bathroom sink)",
    39: "Water (natural stream)",
    40: "Water (ocean waves)",
    41: "Wind through grass",
    42: "Wind and rain storm",
}

df["condition"] = df["condition"].map(map_condition)
df = df[~df.condition.isna()]
df = df[df.snr.isin([-3])]

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)
df.loc[df["model"] == "human", "correct_spkr_list"] = df.loc[df["model"] == "human", "correct_word_list"]

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})
df.loc[df["model"] == "human", "correct_spkr"] = df.loc[df["model"] == "human", "correct_spkr_mean"]
df = df[[
    "condition",
    "snr",
    "correct_word",
    "correct_word_mean",
    "correct_word_sem",
    "correct_spkr",
    "correct_spkr_mean",
    "correct_spkr_sem",
    "model",
    "subject",
]]
df.to_csv(os.path.join(dir_source_data, "fig5e_fig5f.csv"), index=False)
df


,condition,snr,correct_word,correct_word_mean,correct_word_sem,correct_spkr,correct_spkr_mean,correct_spkr_sem,model,subject
0,16mm film projector,-3.0,0.444444,0.380159,0.023234,NaN,NaN,NaN,human,0
1,16mm film projector,-3.0,0.25,0.380159,0.023234,NaN,NaN,NaN,human,1
2,16mm film projector,-3.0,0.222222,0.380159,0.023234,NaN,NaN,NaN,human,2
3,16mm film projector,-3.0,0.5,0.380159,0.023234,NaN,NaN,NaN,human,3
4,16mm film projector,-3.0,0.222222,0.380159,0.023234,NaN,NaN,NaN,human,4
...,...,...,...,...,...,...,...,...,...,...
3436,Wind and rain storm,-3.0,0.484043,0.480585,0.005691,0.880319,0.880319,0.00374,3000,arch0_0007
3437,Wind and rain storm,-3.0,0.470745,0.480585,0.005691,0.880319,0.880319,0.00374,3000,arch0_0008
3438,Wind and rain storm,-3.0,0.50266,0.480585,0.005691,0.87234,0.880319,0.00374,3000,arch0_0009
3439,Wind and rain storm,-3.0,0.470745,0.480585,0.005691,0.875,0.880319,0.00374,3000,arch0_0016


In [13]:
"""
Fig. 6b and 6c (effect of F0 manipulations on word and voice recognition)
"""
df = EXPERIMENT_DATAFRAMES["pitch_altered"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model"}).drop(columns=["tag_expt"])
df = df[df.f0_shift_in_semitones.isin([-12, -9, -6, -3, 0, 3, 6, 9, 12])]

def fix_ragged(dfi):
    if dfi.model == "human":
        n = max(len(dfi["correct_word_list"]), len(dfi["correct_spkr_list"]))
        dfi["subject_list"] = list(range(n))
        while len(dfi["correct_word_list"]) < n:
            dfi["correct_word_list"].append(None)
        while len(dfi["correct_spkr_list"]) < n:
            dfi["correct_spkr_list"].append(None)
    return dfi

df["subject_list"] = None
df = df.apply(add_subject_list, axis=1)
df = df.apply(fix_ragged, axis=1)

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})

df = df[[
    "condition",
    "f0_shift_in_semitones",
    "correct_word",
    "correct_word_mean",
    "correct_word_sem",
    "correct_spkr",
    "correct_spkr_mean",
    "correct_spkr_sem",
    "model",
    "subject",
]]
df.to_csv(os.path.join(dir_source_data, "fig6b_fig6c.csv"), index=False)
df


,condition,f0_shift_in_semitones,correct_word,correct_word_mean,correct_word_sem,correct_spkr,correct_spkr_mean,correct_spkr_sem,model,subject
0,harmonic,-12.0,0.885714,0.779025,0.022284,0.5,0.350766,0.039097,human,0
1,harmonic,-12.0,0.714286,0.779025,0.022284,0.0,0.350766,0.039097,human,1
2,harmonic,-12.0,0.777778,0.779025,0.022284,0.333333,0.350766,0.039097,human,2
3,harmonic,-12.0,0.8,0.779025,0.022284,0.75,0.350766,0.039097,human,3
4,harmonic,-12.0,0.828571,0.779025,0.022284,0.666667,0.350766,0.039097,human,4
...,...,...,...,...,...,...,...,...,...,...
1120,inharmonic,0.0,0.815013,0.797855,0.004715,0.670241,0.655764,0.014853,3000,arch0_0007
1121,inharmonic,0.0,0.820375,0.797855,0.004715,0.552279,0.655764,0.014853,3000,arch0_0008
1122,inharmonic,0.0,0.796247,0.797855,0.004715,0.603217,0.655764,0.014853,3000,arch0_0009
1123,inharmonic,0.0,0.774799,0.797855,0.004715,0.63807,0.655764,0.014853,3000,arch0_0016


In [14]:
"""
Fig. 7c (effect of tone vocoding on speech reception thresholds)
"""
df = EXPERIMENT_DATAFRAMES["hopkins_moore_2009"]
list_k_model = [
    "human",
    "3000",
    "1000",
    "320",
    "50",
]
list_tag_model = [map_k_to_tag_model[k] for k in list_k_model]
df = df[df.tag_model.isin(list_tag_model)].copy()
df["tag_model"] = df["tag_model"].map(map_tag_model_to_k)
df = df.rename(columns={"tag_model": "model", "background_condition": "condition"}).drop(columns=["tag_expt"])
df["condition"] = df["condition"].map({
    3.0: "stationary_noise",
    4.0: "modulated_noise",
})

df["subject_list"] = None
df = df.apply(lambda _: add_subject_list(_, key_metric_list="srt_list"), axis=1)

keys_to_explode = [c for c in df.columns if c.endswith("_list")]
df = df.explode(keys_to_explode).reset_index(drop=True)
df = df.rename(columns={c: c.replace("_list", "") for c in keys_to_explode})

df = df[[
    "condition",
    "cutoff_channel",
    "cutoff_channel_freq",
    "srt",
    "srt_mean",
    "srt_sem",
    "model",
    "subject",
]]
df.to_csv(os.path.join(dir_source_data, "fig7c.csv"), index=False)
df


,condition,cutoff_channel,cutoff_channel_freq,srt,srt_mean,srt_sem,model,subject
0,stationary_noise,0.0,100.0,NaN,-0.700000,0.697452,human,None
1,stationary_noise,8.0,548.0,NaN,-2.047134,0.678344,human,None
2,stationary_noise,16.0,1605.0,NaN,-2.782803,0.898089,human,None
3,stationary_noise,24.0,4102.0,NaN,-2.400637,0.783439,human,None
4,stationary_noise,32.0,10000.0,NaN,-2.992994,0.964968,human,None
...,...,...,...,...,...,...,...,...
725,modulated_noise,32.0,8000.0,-6.495138,-6.852654,0.144758,3000,arch0_0007
726,modulated_noise,32.0,8000.0,-6.659111,-6.852654,0.144758,3000,arch0_0008
727,modulated_noise,32.0,8000.0,-6.545987,-6.852654,0.144758,3000,arch0_0009
728,modulated_noise,32.0,8000.0,-7.485484,-6.852654,0.144758,3000,arch0_0016


In [15]:
"""
Fig. 8c (pure tone frequency discrimination)
"""
regex_fn = "freqdiscriminet/models/freq_quarter_octave/*hz/*IHC*/arch*/results.csv"
df = pd.concat([pd.read_csv(fn).assign(fn=fn) for fn in glob.glob(regex_fn)])

def fn_to_model(fn):
    return [_ for _ in fn.split("/") if "IHC" in _][0]

def fn_to_subject(fn):
    return [_ for _ in fn.split("/") if "arch" in _][0]

df["model"] = df["fn"].map(fn_to_model)
df["subject"] = df["fn"].map(fn_to_subject)
df = df.drop(columns=["log_threshold", "threshold", "fn"])

df_ref = pd.read_csv("freqdiscriminet/data/heinz_etal_2001_fig4a_frequency_discrimination_thresholds.csv")
df_ref["f0"] = df_ref["f0"].astype(np.float32)
df_ref = df_ref[~df_ref["model"].str.contains("human")]
df_ref = df_ref[df_ref["f0"] <= 8e3]
df_ref["subject"] = None
df = pd.concat([df_ref, df]).reset_index(drop=True)
df.to_csv(os.path.join(dir_source_data, "fig8c.csv"), index=False)
df


,f0,weber_fraction,model,subject
0,261.167084,0.000010,all_information_computational,None
1,347.364563,0.000008,all_information_computational,None
2,485.065735,0.000006,all_information_computational,None
3,721.117493,0.000005,all_information_computational,None
4,972.557800,0.000004,all_information_computational,None
...,...,...,...,...
581,8000.000000,0.001589,sr10000_IHC3000,arch_0346_0011
582,8000.000000,0.001791,sr10000_IHC3000,arch_0346_0013
583,8000.000000,0.001830,sr10000_IHC3000,arch_0346_0015
584,8000.000000,0.001821,sr10000_IHC3000,arch_0346_0017
